# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
weather_df=pd.read_csv('../output_data/cities.csv')
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ribeira Grande,38.5167,-28.7000,61.63,64,59,3.29,PT,1651966330
1,Knysna,-34.0363,23.0471,56.98,88,95,2.24,ZA,1651966330
2,Punta Arenas,-53.1500,-70.9167,46.51,66,20,29.93,CL,1651966331
3,Naryan-Mar,67.6713,53.0870,36.16,100,100,12.10,RU,1651966331
4,Waipawa,-41.4122,175.5153,63.27,61,57,21.39,NZ,1651966331
...,...,...,...,...,...,...,...,...,...
545,Jati,24.3539,68.2671,79.23,80,0,11.95,PK,1651966697
546,Ternate,0.8000,127.4000,83.68,75,42,2.82,ID,1651966697
547,Kroonstad,-27.6504,27.2349,52.29,57,50,5.61,ZA,1651966698
548,Kudahuvadhoo,2.6708,72.8944,83.89,78,83,13.24,MV,1651966690


In [44]:
print(weather_df.dtypes)

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


In [45]:
weather_df["Humidity"] = weather_df["Humidity"].astype(float)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [46]:
location = weather_df[["Lat", "Lng"]]
hum_weight = weather_df["Humidity"]

In [47]:
#Choose type of google maps
fig = gmaps.figure(map_type="SATELLITE",center=(25, 0), zoom_level=1.7)
heat_layer = gmaps.heatmap_layer(location, weights=hum_weight, 
                                 dissipating=False,max_intensity=100,
                                 point_radius=2.6)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
nice_weather = pd.DataFrame(weather_df[weather_df["Max Temp"].between(65, 85) & (weather_df['Wind Speed'] <= 15) & 
                                       (weather_df["Cloudiness"] <30)& (weather_df["Humidity"]<= 60)&(weather_df["Humidity"]>= 25)]) 
nice_weather.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,2.300000e+01
mean,18.758074,18.782613,72.564348,42.608696,6.478261,8.554783,1.651967e+09
std,26.086398,81.364244,5.730005,10.426100,8.831090,4.640337,8.674776e+01
min,-40.813500,-121.423800,65.120000,28.000000,0.000000,0.000000,1.651966e+09
25%,16.015250,-63.376750,67.300000,32.500000,0.000000,5.415000,1.651966e+09
50%,29.266600,44.630100,73.470000,44.000000,1.000000,8.950000,1.651967e+09
75%,36.077150,68.651950,76.910000,51.500000,10.500000,12.605000,1.651967e+09
max,42.241200,139.500000,84.070000,59.000000,28.000000,14.970000,1.651967e+09


In [49]:
nice_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,Dothan,31.2232,-85.3905,80.80,48.0,0,14.97,US,1651966395
136,Viedma,-40.8135,-62.9967,67.60,36.0,12,14.97,AR,1651966427
172,Shingū,33.7333,135.9833,70.02,50.0,8,1.72,JP,1651966446
231,General Pico,-35.6566,-63.7568,66.16,49.0,28,12.55,AR,1651966489
232,Sur,22.5667,59.5289,84.07,57.0,0,7.40,OM,1651966489


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
# Add column for hotel data 
hotel_df= nice_weather
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,Dothan,31.2232,-85.3905,80.80,48.0,0,14.97,US,1651966395
136,Viedma,-40.8135,-62.9967,67.60,36.0,12,14.97,AR,1651966427
172,Shingū,33.7333,135.9833,70.02,50.0,8,1.72,JP,1651966446
231,General Pico,-35.6566,-63.7568,66.16,49.0,28,12.55,AR,1651966489
232,Sur,22.5667,59.5289,84.07,57.0,0,7.40,OM,1651966489


In [51]:
#Add extra column
hotel_df["Hotel Name"] = " "
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
94,Dothan,31.2232,-85.3905,80.80,48.0,0,14.97,US,1651966395,
136,Viedma,-40.8135,-62.9967,67.60,36.0,12,14.97,AR,1651966427,
172,Shingū,33.7333,135.9833,70.02,50.0,8,1.72,JP,1651966446,
231,General Pico,-35.6566,-63.7568,66.16,49.0,28,12.55,AR,1651966489,
232,Sur,22.5667,59.5289,84.07,57.0,0,7.40,OM,1651966489,


In [52]:
target_type = "lodging"
coords = location
radius = 5000
params = {
    "location": coords,
    "types": target_type,
    "radius": radius,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print("Search for hotels near cities:")

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    
    params["location"] = f"{lat},{lng}"


    # request and print url
    hotel_response = requests.get(base_url, params=params)
    #Converting to json
    hotel_name = hotel_response.json()
    

    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        print(f'In {hotel_name["City"]}, you can find the{hotel_name["results"][0]["name"]}')
        #hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Skipping.")


Search for hotels near cities:
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.
Skipping.


In [53]:
hotel_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
94,Dothan,31.2232,-85.3905,80.80,48.0,0,14.97,US,1651966395,
136,Viedma,-40.8135,-62.9967,67.60,36.0,12,14.97,AR,1651966427,
172,Shingū,33.7333,135.9833,70.02,50.0,8,1.72,JP,1651966446,
231,General Pico,-35.6566,-63.7568,66.16,49.0,28,12.55,AR,1651966489,
232,Sur,22.5667,59.5289,84.07,57.0,0,7.40,OM,1651966489,
254,Mount Isa,-20.7333,139.5000,67.77,45.0,0,9.22,AU,1651966510,
298,Borazjan,29.2666,51.2159,80.47,34.0,4,3.56,IR,1651966528,
325,Kashgar,39.4547,75.9797,67.21,32.0,0,8.95,CN,1651966548,
327,Laguna,38.4210,-121.4238,77.95,32.0,0,5.75,US,1651966549,
339,Koson,39.0375,65.5850,70.12,44.0,7,8.16,UZ,1651966556,


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
     info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))